#### This program will function as an LLM Tutor utilizing the OpenAI API

Import Statements

In [15]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

Load OpenAI API Key

In [16]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


#### Setting up the Environment

System Prompt

In [17]:
# The system prompt is what explains the context of the situation to the frontier model
# It tells them what kind of task they are performing and what tone to use

system_prompt = "You are provided with a question.. \
    You are going to best answer the question as \
    if you are a tutor of the relevant subject matter.\n"
system_prompt += "You should respond with a normal conversational prose"

User Prompt

In [18]:
# A function that writes a User Prompt that asks for summaries of websites:
# The user prompt is the actual conversation itself
# The converstaion start and the role of the LLM is to figure out what way to respond to the user prompt in the context of the system prompt

def user_prompt_for(question):
    user_prompt = f"You are looking at a question {question}"
    user_prompt += "\nThis question is about how to understand a piece of code; \
please provide a walkthrough on what this code is doing.\n\n"
    user_prompt += question
    return user_prompt

Define Answer for Question Function

In [19]:
# Create a messages list using the same format that we used for OpenAI

def answer_for_question(question):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(question)}
    ]

Define Question List

In [20]:
question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

Answer Question Function

In [21]:
# And now: call the OpenAI API. You will get very familiar with this!

def answer_question(question):
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = question
    )
    return response.choices[0].message.content

Displaying Answer Function

In [22]:
# A function to display this nicely in the Jupyter output, using markdown

def display_answer(question):
    # Get the structured prompt for the question
    structured_prompt = answer_for_question(question)
    # Generate the answer using the appropriate function
    answer = answer_question(structured_prompt)
    # Display the answer as Markdown
    display(Markdown(answer))


In [23]:
display_answer(question)

Sure! Let's break down the code step by step. 

The code you provided is a Python expression that utilizes a generator function through the `yield from` statement, alongside a set comprehension. Here’s a walk-through of what each part does:

1. **Set Comprehension**: 
   The expression `{book.get("author") for book in books if book.get("author")}` is a set comprehension. This constructs a set of unique authors from a collection called `books`. 
   
   - `for book in books`: This loops over each item in the collection `books`.
   - `book.get("author")`: This tries to retrieve the value associated with the key "author" from each `book` dictionary (assuming each item in `books` is a dictionary).
   - `if book.get("author")`: This conditional ensures that only those authors that are defined (not `None` or empty) are included. If an author is not present, `book.get("author")` will return `None`, and that will not be added to the set.

2. **Yield From**:
   The keyword `yield` is used in Python to turn a function into a generator, which can yield results one at a time and pause its execution. The `yield from` statement is a way to yield all values from an iterable, in this case, the set produced by the comprehension. 

   So in simpler terms, `yield from ...` takes each item from the set of authors and yields it one by one. 

3. **Overall Effect**:
   The entire line of code constructs a set of unique authors from the `books` list and yields each author in turn. This could be useful, for example, if you want to create a function that processes or displays each unique author from a collection of books.

**Why Use This?**:
- **Uniqueness**: Using a set ensures that there are no duplicate authors. If the same author appears in multiple `book` entries, they will only be yielded once.
- **Conditional Filtering**: The conditional inside the comprehension effectively filters out books without an author, making your output cleaner and more meaningful.
- **Efficiency**: By using a generator, it allows you to process authors one at a time, which can be more memory efficient especially if `books` is a large collection.

In summary, this piece of code is a compact and efficient way to extract and yield unique authors from a collection of books while filtering out any entries that lack an author.

Streaming the result from OpenAI

In [24]:
def stream_answer(question):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt_for(question)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [25]:
stream_answer(question)

Sure! Let's break down the piece of code you've presented:

python
yield from {book.get("author") for book in books if book.get("author")}


### 1. **Contextual Setup**
This code snippet appears to be part of a generator function. The `yield from` statement is used in Python to yield all values from an iterable (like a list, set, or another generator). In this case, it's yielding values from a set comprehension.

### 2. **The Set Comprehension**
Inside the `yield from`, we have a set comprehension:

python
{book.get("author") for book in books if book.get("author")}


Breaking this down:

- **`for book in books`**: This iterates over a collection called `books`, where each `book` is likely a dictionary representing a book's details (like title, author, year, etc.).

- **`book.get("author")`**: This method is called on the dictionary `book`. The `get` method is used to retrieve the value associated with the key `"author"`. If the key is not present, it returns `None`.

- **`if book.get("author")`**: This clause filters out any books that do not have an author. Essentially, if `book.get("author")` returns a valid author name (not `None` or empty), that author's name will be included in the set.

### 3. **The Resulting Set**
The result of the set comprehension is a set of unique author names. Using a set is beneficial here because it automatically removes any duplicate author names — if two or more books have the same author, that author's name will appear only once in the set.

### 4. **Using yield from**
After creating this set of authors, the `yield from` expression will yield each author in the set one by one. This allows the generator to provide a sequence of author names rather than returning the entire set at once.

### 5. **Why Use This Code?**
- **Efficiency**: The use of `yield from` generates values on the fly, which can be more memory-efficient compared to creating a complete list or set in memory before returning.

- **Uniqueness**: The choice of a set instead of a list ensures that you don't have duplicate authors in the output.

- **Readability**: This concise syntax is often more readable and expressive, clearly demonstrating the intention to gather authors from a collection of books.

### Summary
In summary, this code yields each unique author from a collection of books, skipping any books without an author. It efficiently collects author names into a set and returns them one at a time through a generator. This approach captures the essence of Python's expressive capabilities regarding collections and generators!